#### Normalization(정규화)
- 입력 데이터를 정규화 한다 = 모든 입력 변수가 비슷한 범위와 분포를 갖도록 조정하는 것.
- 모델은 각 입력 변수의 중요성을 적절히 반영하여 좀 더 정확환 예측을 할 수 있게됨.
- 깊은 층을 쌓을 수록 분포의 차이가 발생할 가능성이 높아지고, 학습이 잘 되지 않는다. 
    - 딥러닝 분야에서는 이를 막기위해 층과 층 사이에 정규화를 추가해 학습을 안정적으로 만드는 기법을 사용해왔음.

##### 딥러닝에서 배치 정규화 / 층 정규화
- 평균과 표준편차를 구할 데이터를 어떻게 묶는지에 따라 나뉨.
    - 일반적으로 이미지 처리에는 배치 정규화를, 자연어 처리에는 층 정규화를 사용함.
- 자연어 처리에서 배치 정규화를 사용하지 않는 이유?
    -> 입력으로 들어가는 문장의 길이가 다양한데, 배치 정규화를 사용할 경우 정규화에 포함되는 데이터의 수가 제각각이라 정규화 효과를 보장하기 어렵기 때문.  
    -> 이런 단점을 보완할 수 있도록 각 토큰 임베딩의 평균과 표준편차를 구해 층 정규화를 수행.

##### 사후 정규화(post-norm) / 사전 정규화(pre-norm)
- 사후 정규화: attention과 feed foward 층 이후에 층 정규화를 적용
- 사전 정규화: 먼저 층 정규화를 적용한 후, attention과 feed foward 층 적용
-> 사전 정규화가 더 안정적인 학습

In [1]:
# 띄어쓰기 단위로 분리하는 토큰화 코드
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids:  [0, 1, 2, 3, 4]


In [2]:
# 토큰 임베딩 (토큰 아이디 -> 벡터)
import torch
import torch.nn as nn

embedding_dim = 16
# 사전 크기가 ?이고, 차원이 ?인 embed_layer 생성
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

# 입력 토큰을 임베딩으로 변환
# 즉, 토큰 1개를 16차원의 벡터로 변환
input_embeddings = embed_layer(torch.tensor(input_ids))     # (5, 16)
input_embeddings = input_embeddings.unsqueeze(0)        #(1, 5, 16)
input_embeddings.shape

torch.Size([1, 5, 16])

In [ ]:
# 층 정규화 코드

import torch.nn as nn

# 층 정규화 layer 생성
norm = nn.LayerNorm(embedding_dim)
# 정규화된 embedding
norm_x = norm(input_embeddings)
print(norm_x.shape)

# 실제 평균과 표준편차 확인
norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data

torch.Size([1, 5, 16])


(tensor([[0.0000e+00, 4.4703e-08, 1.4901e-08, 0.0000e+00, 2.6077e-08]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

##### Feed forward layer
- 데이터의 특징을 학습하는 Fully Connected Layer를 말함.
- 입력 텍스트의 전체를 이해하는 역할을 담당함.
- linear, dropout, layernorm, activation function으로 구성됨.
- 입출력의 형태가 동일해야 쉽게 층을 쌓아 확장이 가능하기 때문에, 동일하게 맞춤.
    - 일반적으로, d_model차원에서 d_model보다 2~3배 큰 dim_feedfoward 차원으로 확장했다가 다시 d_model로 변환함.

In [8]:
# feed foward layer
class PreLayerNormFeedFoward(nn.Module):
    def __init__(self, d_model, dim_feedforward, dropout):
        super().__init__()
        # 첫 번째 linear layer
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        # 두 번째 linear layer
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        # 첫 번째 dropout layer
        self.dropout1 = nn.Dropout(dropout)
        # 두 번째 dropout layer
        self.dropout2 = nn.Dropout(dropout)
        # 활성화 함수
        self.activation = nn.GELU()
        # 층 정규화
        self.norm = nn.LayerNorm(d_model)

    def forward(self, src):
        x = self.norm(src)
        x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
        x = self.dropout2(x)
        return x

In [9]:
pre_ffn = PreLayerNormFeedFoward(embedding_dim, 48, 0.5)  # 클래스 초기화
output = pre_ffn(input_embeddings)  # forward 연산 수행
print(output)

tensor([[[ 0.0000, -0.0000, -0.2623, -0.0000,  2.4352,  0.0866,  0.0000,
          -1.3436, -0.0000, -1.8153, -0.0000,  0.0000,  0.0000,  1.6701,
          -0.0000, -1.0277],
         [ 1.3755, -0.0000, -0.0000, -0.0000,  0.0000,  3.9125, -0.0000,
           0.0000, -0.0000, -0.0000,  0.0000, -0.6045,  0.0000,  0.0000,
          -0.0000, -0.0000],
         [-0.0000, -0.3086,  0.0898, -0.3385,  0.0000, -0.0000,  0.0000,
           3.3179,  2.4504, -2.8544, -0.5796, -0.6185, -0.0000, -0.0000,
          -1.3778,  1.9032],
         [-4.2328,  0.6848,  0.0000,  0.0000, -0.0000, -0.0000,  0.0000,
           0.0000,  0.0000, -0.0000,  0.0000, -1.5486, -1.8566, -0.9946,
          -0.5051, -1.1509],
         [ 0.0000, -2.4412,  1.6680, -1.1291,  0.0000,  3.1412,  0.0000,
           2.9621, -1.7284, -0.5575, -0.0000, -0.0000,  0.0000,  0.0000,
          -0.0000,  0.2266]]], grad_fn=<MulBackward0>)
